In [ ]:
# Parameters (do not containerize this cell)
param_data_filename <- "Template_MBO_Example_raw.xlsx"
param_metadata_sheet <- "METADATA"
param_data_sheet <- "BIRDS"
param_user_name <- "See URL: beta.naavre.net/jupyter/user/[param_user_name]/lab"
param_use_dummy_data <- "true"
param_years <- 7
param_latitude_north <- 90.0000
param_latitude_south <- 25.0000
param_longitude_east <- 70.0000
param_longitude_west <- 00.0000
param_upper_limit_max_depth <- 0
param_lower_limit_max_depth <- 50
param_upper_limit_min_depth <- 0
param_lower_limit_min_depth <- 50
param_first_month <- 1
param_last_month <- 3
param_output_samples_ecological_parameters <- "false"
param_make_plots <- "true"
param_transform_to_log10 <- "true"
conf_temporary_data_directory <- "/tmp/data"
conf_virtual_lab_biotisan_euromarec <- "vl-biotisan-euromarec"
conf_minio_endpoint <- "scruffy.lab.uvalight.net:9000"
conf_minio_region <- "nl-uvalight"
conf_minio_public_bucket <- "naa-vre-public"
conf_minio_user_bucket <- "naa-vre-user-data"

In [ ]:
# Secrets (do not containerize this cell)
library("SecretsProvider")

secretsProvider <- SecretsProvider()
secret_minio_access_key = ""
secret_minio_access_key = secretsProvider$get_secret("secret_minio_access_key")
secret_minio_secret_key = ""
secret_minio_secret_key = secretsProvider$get_secret("secret_minio_secret_key")

In [ ]:
# Species occurrence data cleaner
library(dplyr)
library(tidyr)

print(paste(deparse(substitute(number_of_validation_errors)), number_of_validation_errors, sep=" = "))

validate_dataframe_has_data <- function(dataframe, dataframe_name) {
    if (nrow(dataframe) == 0) {
    stop(paste0(dataframe_name, " has no rows (0 rows). Halting execution."))
  } else {
    sprintf("%s has %i rows.", dataframe_name, nrow(dataframe))
    }
}

# Report on parameters set
# print(sprintf("Filtering sites with %i or more years of data.", param_years))
# print(sprintf("Filtering for data within the coordinates %s south, %s north, %s east and %s west.", param_latitude_south, param_latitude_north, param_longitude_east, param_longitude_west))
# print(sprintf("Filtering for data within upper_limit_max_depth %i, lower_limit_max_depth %i, upper_limit_min_depth %i, lower_limit_min_depth %i.", param_upper_limit_max_depth, param_lower_limit_max_depth, param_upper_limit_min_depth, param_lower_limit_min_depth))
# print(sprintf("Filtering for data between the month %i and month %i.", param_first_month, param_last_month))

# Assign dummy variables to prevent false Input/Output detection by the NaaVRE cell analyzer
datecollected = ""
siteid = ""
decimallatitude = ""
decimallongitude = ""

# Read (meta)data from files
md <- read.csv(paste(conf_temporary_data_directory, metadata_as_csv_filename, sep="/"), sep=",")
data <- read.csv(paste(conf_temporary_data_directory, data_as_csv_filename, sep="/"), sep=",")
validate_dataframe_has_data(data, "data read from csv")
validate_dataframe_has_data(md, "metadata read from csv")

###### Years filter ######
###### The "number of sampled years" could be changed by the user (default = 7) ######
# Create a table with sites with more than 7 sampling years
sites <- data %>% 
  group_by(siteid) %>%
  summarise(nyear = n_distinct(substr(datecollected, 1, 4))) %>%
  filter(nyear > param_years)

md <- merge(md,sites, by = "siteid")
validate_dataframe_has_data(md, "metadata merged by siteid")
validate_dataframe_has_data(data, "data after first years filter")

###### Coordinates filter ######
###### The "geographical boundaries" could be changed by the user (default = latitude (25:90), longitude (-45:70)). The default values correspond to European continental waters. ######
# Keep sites within the study area [our boundaries are latitude (25:90), longitude (-45:70)]
metadata_coordinates <- md %>% select(siteid, decimallatitude, decimallongitude)
print(metadata_coordinates)

md <- dplyr::filter(md, decimallatitude >= param_latitude_south, decimallatitude <= param_latitude_north, 
                 decimallongitude >= param_longitude_west, decimallongitude <= param_longitude_east)
print(paste0("Number of sites found within the specified geolocation: ", nrow(md)))

data <- data %>% # Keep data from these sites
  filter(siteid %in% md$siteid)
validate_dataframe_has_data(data, "data filtered by coordinates")

###### Depth filter ######
###### The "depth" could be changed by the user (default = 0 to 50 meters). It should be given in absolute value. ######
#(in this case is not necessary, but for other taxonomic groups is possible that the sample were taken at different depths. The code keeps the NAs in case that information is not known)
data <- data %>% filter((maximumdepthinmeters >= param_upper_limit_max_depth & maximumdepthinmeters <= param_lower_limit_max_depth) %>% tidyr::replace_na(TRUE))
data <- data %>% filter((minimumdepthinmeters >= param_upper_limit_min_depth & minimumdepthinmeters <= param_lower_limit_min_depth) %>% tidyr::replace_na(TRUE))
validate_dataframe_has_data(data, "data filtered by depth")

###### Season filter ######
###### The "season" could be changed by the user (default = 1:3). The default values correspond to winter.
# The period does not need to be three months, can be 1:12 for the whole year. It cannot choose months from different years (for example, November to January) ######
# In this case, most of the sampling campaigns were conducted in winter
# One was conducted in summer and should be removed since the sampling season is not consistent
data$month <- as.numeric(format(as.Date(data$datecollected), "%m")) # Create a column with the sampling month

season <- c(param_first_month:param_last_month)
data <- data %>%
  filter(month %in% season) #Remove those samples in non-consistent seasons (in this case keeps the months 1, 2 and 3, this is January, February and March)
validate_dataframe_has_data(data, "data filtered by season")

# Note that some time series can have more than one sampling campaign per year and even per season (not in this case)
# For our analysis, we are only keeping one sampling campaign per year

###### Years filter ######
###### Note that this step is repeated ######
# Update the table with sites with more than the number of sampled years
# After removing inconsistent sampling campaigns, some time series may become shorter than 8 years
sites <- data %>% 
  group_by(siteid) %>%
  summarise(nyear = n_distinct(substr(datecollected, 1, 4))) %>%
  filter(nyear > param_years)

data <- data %>% # Keep data from these sites
  filter(siteid %in% md$siteid)
md <- md %>% # Keep metadata from these sites
  filter(siteid %in% md$siteid)
validate_dataframe_has_data(data, "data filtered by years")

md_final <- md[,c(1:8)]
data_final <- data[,c(1:15)]

# Write data to files
cleaned_metadata_filename <- "metadata_Example.csv"
cleaned_data_filename <- "data_Example.csv"
cleaned_metadata_path <- paste(conf_temporary_data_directory, cleaned_metadata_filename, sep="/")
cleaned_data_path <- paste(conf_temporary_data_directory, cleaned_data_filename, sep="/")
print(sprintf("Storing metadata in: %s, and data in %s", cleaned_metadata_path, cleaned_data_path))
write.csv(md_final, file = cleaned_metadata_path)
write.csv(data_final, file =  cleaned_data_path)